<a href="https://colab.research.google.com/github/shihchun/ML-homework/blob/master/CNN_handwritting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 手寫 辨識 Mnist dataset

[感知機 source](https://colab.research.google.com/drive/1wAJAcsal8SiEW7lkRiBSquv-EnRGTdxJ)

### 介紹
- CNN範例架構圖

<img src="https://i.imgur.com/lgukCcA.png" height="300">

- Mnist Dataset test image

<img src="https://i.imgur.com/sI16NtN.png" height="300">

- Mnist dataset test label output

<img src="https://i.imgur.com/OBRp6Gp.png" height="300">

- 結果的編碼方式 one＿hot

```
# [0,9] 的十位數字 (one hot = True 的編碼處理) 大概了解即可
# 0: 1000000000
# 1: 0100000000
# ......
```

<img src="https://i.imgur.com/7VHEjKe.png" height="300">

- 影像處理補零

<img src="https://i.imgur.com/7W6LP9Q.png" height="300">


In [0]:
import tensorflow as tf
import numpy as np

# 加入手寫數字庫
from tensorflow.examples.tutorials.mnist import input_data

# 儲存在根目錄（sys.path）下面的mnist_data的目錄，one_hot （一種編碼方式）
mnist = input_data.read_data_sets("mnist_data/", one_hot=True) # [55000,28,28] 55000張圖片可以訓練
print('mnist.test.images：', np.shape(mnist.test.images))
print('mnist.test.labels：', np.shape(mnist.test.labels))
test_x = mnist.test.images[:3000] # 抓下 0～2999 三千張圖和標籤
test_y = mnist.test.labels[:3001]


# [0,9] 的十位數字 (one hot = True 的編碼處理) 大概了解即可
# 0: 1000000000
# 1: 0100000000
# ......


# None 表示張量（tensor）的第一個維度可以是任何長度
input_x = tf.placeholder(tf.float32, [None, 28*28]) / 255 # 輸入28x28x1
output_y = tf.placeholder(tf.int32, [None,10]) # 輸出1x1x10 
input_x_images = tf.reshape(input_x, [-1,28,28,1]) # 改變train_x shape to [28,28,1]

# conv2d 有 tf.nn.conv2d（輸入tensor，float32或是half）、tf.layer.conv2d（輸入tensor，底層使用nn做的）
# 卷積神經網路 
# 第一層 28x28x1 -> 28x28x32 conv濾波器 32個
conv1 = tf.layers.conv2d(
    inputs = input_x_images, # [28,28,1]
    filters = 32, # input int
    kernel_size = [5, 5], # mask size輸入5x5
    strides = 1, # 遮罩相隔多少距離去採樣（輸入list或是tuple元組）
    padding = 'same' # 補零方案外圍兩圈（採樣超過28x28的時候，valid不會採樣超過，same表示採樣後大小相同28x28x1）
) # 輸出[28,28,32]
# 池化層 抓出較大的值率出雜訊，避免判斷會出現問題
pool1 = tf.layers.max_pooling2d(
    inputs = conv1, #[28,28,32]
    pool_size = [2, 2], # 濾波器 2*2
    strides = 2 # 相隔2採樣一次，大小會變成0.5倍
) # 輸出[14,14,32]



# 層數加多 提取更多特徵
# 第二層 14,14,32 -> 14x14x64 conv濾波器 64個
conv2 = tf.layers.conv2d(
    inputs = pool1, # [14,14,32]
    filters = 64, # input int
    kernel_size = [5, 5], # mask size輸入5x5
    strides = 1, # 遮罩相隔多少距離去採樣（輸入list或是tuple元組）
    padding = 'same', # 補零方案外圍兩圈（採樣超過28x28的時候，valid不會採樣超過，same表示採樣後大小相同28x28x1）
) # 輸出[14,14,64]
# 池化層 抓出較大的值率出雜訊，避免判斷會出現問題
pool2 = tf.layers.max_pooling2d(
    inputs = conv2, #[14,14,64]
    pool_size = [2, 2], # 濾波器 2*2
    strides = 2, # 相隔2採樣一次，大小會變成0.5倍
) # 輸出[7,7,64]

# do flat
flat = tf.reshape(pool2,[-1, 7*7*64]) # 變成[7*7*64, ]

# 1024個神經元 全連結層 fully connected
dense = tf.layers.dense(
    inputs = flat, # input tensor = flat [7*7*64, ]
    units =1024, # unit = 1024
    activation = tf.nn.relu # 激發函數 非線性函數線性化
    )

# Droupout : 50% 0.5
dropout = tf.layers.dropout(inputs = dense, rate =0.5) # 1x1x10

# 10 個神經元 全連結層 fully connected 不用激發函數
logits = tf.layers.dense(inputs = dropout, units = 10) # 輸出 1x1x10

# 計算 loss function
loss = tf.losses.softmax_cross_entropy(
    onehot_labels = output_y,
    logits = logits
)

# Adam 最佳化（最小化誤差） 設定 學習率\yida = 0.001
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# 計算 預測結果 和 實際標籤 匹配程度 summary
# return (accuracy, update_op) 是局部變量： 是擁有局部作用域的變量
accuracy = tf.metrics.accuracy(
    labels=tf.argmax(output_y, axis=1),
    predictions=tf.argmax(logits, axis=1),
    )[1] # 返回 第二個 update_op

# 建立 session
sess = tf.Session()

# 初始化 ini #局部變量要另外加進來
init = tf.group(
    tf.global_variables_initializer(), 
    tf.local_variables_initializer() # 局部變量(accuracy, update_op)
    )
sess.run(init)

for i in range(100):
  batch = mnist.train.next_batch(50) #從 mnist.train 找到下面50個樣本
  train_loss, train_op_ = sess.run(
      [loss, train_op], 
      {input_x: batch[0], output_y: batch[1]}
      )
  test_accuracy = sess.run(accuracy, {input_x: test_x, output_y:test_y})
  print( "Step=%d, Train loss=%.4f, [Test accuracy=%.2f]" 
            % (i, train_loss, test_accuracy)
  )
  pass

# print 比對
test_output = sess.run(logits, {input_x: test_x[:20]})
inferenced_y = np.argmax(test_output, 1)
print(inferenced_y, 'Inferenced numbers 推測數字')

  

Extracting mnist_data/train-images-idx3-ubyte.gz
Extracting mnist_data/train-labels-idx1-ubyte.gz
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
mnist.test.images： (10000, 784)
mnist.test.labels： (10000, 10)
Step=0, Train loss=2.2977, [Test accuracy=0.09]
Step=1, Train loss=2.3349, [Test accuracy=0.20]
Step=2, Train loss=2.0275, [Test accuracy=0.30]
Step=3, Train loss=1.9048, [Test accuracy=0.35]
Step=4, Train loss=1.5380, [Test accuracy=0.39]
Step=5, Train loss=1.4812, [Test accuracy=0.41]
Step=6, Train loss=1.1846, [Test accuracy=0.45]
Step=7, Train loss=1.1024, [Test accuracy=0.48]
Step=8, Train loss=0.7830, [Test accuracy=0.51]
Step=9, Train loss=0.5993, [Test accuracy=0.53]
Step=10, Train loss=0.8900, [Test accuracy=0.55]
Step=11, Train loss=0.4047, [Test accuracy=0.57]
Step=12, Train loss=0.7705, [Test accuracy=0.59]
Step=13, Train loss=0.4767, [Test accuracy=0.60]
Step=14, Train loss=0.4645, [Test accuracy=0.61]
Step=15, Train los